# HPS Diameter Distribution Reproducibility

This notebook orchestrates the end-to-end reproduction of the horizontal point sampling (HPS) distribution fitting experiment. Run all cells to regenerate figures, tables, and inspect parameter fits.


## 1. Environment Setup

Install the project requirements if needed (skip inside managed environments).


In [ ]:
!python -m pip install -r ../requirements.txt

## 2. Load Processed Dataset

The notebook expects the pre-binned meta-plot dataset in `../data/processed/binned_meta_plots.parquet`.


In [ ]:
from pathlib import Path
import pandas as pd

data_path = Path("../data/processed/binned_meta_plots.parquet")
if not data_path.exists():
    raise FileNotFoundError("Processed dataset missing; run preprocessing to create it.")

binned = pd.read_parquet(data_path)
binned.head()


## 3. Fit Distributions

Compare the control (size-biased) and test (weighted) methods on each meta-plot.


In [ ]:
from scripts import fitting
import pandas as pd

results = []
for (species, cover), df in binned.groupby(["species_group", "cover_type"]):
    x = df["dbh_cm"].to_numpy()
    tally = df["tally"].to_numpy()
    for dist_name in ("weibull", "gamma"):
        dispatch = fitting.method_dispatch(dist_name)
        control = dispatch["control"](x, tally)
        test = dispatch["test"](x, tally)
        results.append({
            "species_group": species,
            "cover_type": cover,
            "distribution": dist_name,
            "rss_control": control.rss,
            "rss_test": test.rss,
        })

summary = pd.DataFrame(results)
summary


## 4. Regenerate Figures & Tables

Call the command-line scripts to rebuild manuscript artefacts.


In [ ]:
!python ../scripts/generate_figures.py --config ../config/figures.yml

In [ ]:
!python ../scripts/generate_tables.py --config ../config/tables.yml